In [2]:
import sympy as sym
import numpy as np
from scipy.optimize import direct, Bounds, shgo
from dreal import *

import torch
import sympytorch

In [3]:
x1, x2 = sym.symbols("x1, x2")

x1_deri = x2
x2_deri = -x1 - (1 - x1 ** 2) * x2


def final_check(sympy_expr):
    x1 = sym.symbols('x1')
    x2 = sym.symbols('x2')

    x3, x4, x5, x6 = sym.symbols('x3, x4, x5, x6')

    x7, x8, x9, x10 = sym.symbols("x7, x8, x9, x10")

    origin = float(sympy_expr.evalf(subs = {"x1": 0, "x2": 0}))
    sympy_expr = sympy_expr - origin
    # sympy_expr = sympy_expr.subs(x3, 9.81)
    numpy_expr = sym.lambdify((x1,x2), sympy_expr, "numpy")
    v_dot = -1 * (sympy_expr.diff(x1) * x1_deri + sympy_expr.diff(x2) * x2_deri)
    numpy_v_dot = sym.lambdify((x1,x2), v_dot, "numpy")

    
    # whole state random check
    global function_check_1
    global function_check_2

    def function_check_1(x):
        return numpy_expr(x[0], x[1])
    
    def function_check_2(x):
        return numpy_v_dot(x[0], x[1])
    
    

    num_points = 10 ** 7
    # pool_1 = None
    for i in range(5):
        # data = np.random.uniform(-1.0, 1.0, (1 * 10 ** 7, 6))

        boundary_points = []

        # Iterate over each dimension to create points near the boundaries
        for j in range(2):
            # Points near the -1 boundary for the i-th dimension
            point_set_low = np.random.uniform(-1.0, 1.0, (num_points, 2))
            point_set_low[:, j] = -1 + np.abs(0.001 * np.random.rand(num_points))
    
            # Points near the 1 boundary for the i-th dimension
            point_set_high = np.random.uniform(-1.0, 1.0, (num_points, 2))
            point_set_high[:, j] = 1 -  np.abs(0.001 * np.random.rand(num_points))
    

            point_uniform = np.random.uniform(-1.0, 1.0, (num_points,2))
            # Add these points to the boundary_points list
            boundary_points.append(point_set_low)
            boundary_points.append(point_set_high)
            boundary_points.append(point_uniform)

        # Combine all boundary points into one array
        boundary_points = np.vstack(boundary_points)

        check_1 = function_check_1(boundary_points)
        check_2 = function_check_2(boundary_points)

        if any(check_1 < 0) or any(check_2 < 0):
            return False
        
    return True

def pgd_attack(
    x0, f, eps, steps=10, lower_boundary=None, upper_boundary=None, direction="minimize"
):
    """
    Use adversarial attack (PGD) to find violating points.
    Args:
      x0: initialization points, in [batch, state_dim].
      f: function f(x) to find the worst case x to maximize.
      eps: perturbation added to x0.
      steps: number of pgd steps.
      lower_boundary: absolute lower bounds of x.
      upper_boundary: absolute upper bounds of x.
    """
    # Set all parameters without gradient, this can speedup things significantly
    grad_status = {}
    try:
        for p in f.parameters():
            grad_status[p] = p.requires_grad
            p.requires_grad_(False)
    except:
        pass

    step_size = eps / steps * 2
    noise = torch.randn_like(x0) * step_size
    if lower_boundary is not None:
        lower_boundary = torch.max(lower_boundary, x0 - eps)
    else:
        lower_boundary = x0 - eps
    if upper_boundary is not None:
        upper_boundary = torch.min(upper_boundary, x0 + eps)
    else:
        upper_boundary = x0 + eps
    x = x0.detach().clone().requires_grad_()
    # Save the best x and best loss.
    best_x = torch.clone(x).detach().requires_grad_(False)
    fill_value = float("-inf") if direction == "maximize" else float("inf")
    best_loss = torch.full(
        size=(x.size(0),),
        requires_grad=False,
        fill_value=fill_value,
        device=x.device,
        dtype=x.dtype,
    )
    for i in range(steps):
        output = f(x1 = x[:,[0]], x2 = x[:,[1]]).squeeze(1).squeeze(1)
        # output = torch.clamp(f(x).squeeze(1), max=0)
        output.mean().backward()
        if direction == "maximize":
            improved_mask = output >= best_loss
        else:
            improved_mask = output <= best_loss
        best_x[improved_mask] = x[improved_mask]
        best_loss[improved_mask] = output[improved_mask]
        # print(f'step = {i}', output.view(-1).detach())
        # print(x.detach(), best_x)
        noise = torch.randn_like(x0) * step_size / (i + 1)
        if direction == "maximize":
            x = (
                (
                    torch.clamp(
                        x + torch.sign(x.grad) * step_size + noise,
                        min=lower_boundary,
                        max=upper_boundary,
                    )
                )
                .detach()
                .requires_grad_()
            )
        else:
            x = (
                (
                    torch.clamp(
                        x - torch.sign(x.grad) * step_size + noise,
                        min=lower_boundary,
                        max=upper_boundary,
                    )
                )
                .detach()
                .requires_grad_()
            )

    # restore the gradient requirement for model parameters
    try:
        for p in f.parameters():
            p.requires_grad_(grad_status[p])
    except:
        pass
    return best_x

def pgd_check(sympy, X):

    if len(list(sympy.free_symbols)) < 2:
        return False, np.array([])

    x1, x2, x3, x4, x5, x6, x7, x8 = sym.symbols('x1, x2, x3, x4, x5, x6, x7, x8')

    sympy = -1 * (sympy.diff(x1) * x1_deri + sympy.diff(x2) * x2_deri)

    if len(list(sympy.free_symbols)) < 1:
        return False, np.array([])
    
    sympy_torch = sympytorch.SymPyModule(expressions=[sympy]).to("cuda:1")
    data = torch.rand((X.shape[0], X.shape[1])).to("cuda:1")
    result = pgd_attack(data, sympy_torch, 0.8, steps=30, lower_boundary=torch.tensor([-1.0, -1.0]).to("cuda:1"), upper_boundary=torch.tensor([1.0, 1.0]).to("cuda:1"), direction="minimize")
    result = torch.clamp(result, -1.0, 1.0)
    eval = sympy_torch(x1 = result[:,[0]], x2 = result[:,[1]]).squeeze(1).squeeze(1)
    result = result[eval < - 0]

    return (len(result) == 0), result.cpu().detach().numpy()

In [4]:
final_check(x1 ** 2 + x2 ** 2)

True

In [7]:
pgd_check(x1 ** 2 + x2 ** 2, torch.ones((1000000, 2)))

(True, array([], shape=(0, 2), dtype=float32))

In [15]:
def derivative_calculate(v, x1, x2):

    
    # x1_deri = -x1 + 0.5 * x2 - 0.1 * x5 ** 2
    # x2_deri = -0.5 * x1 - x2
    # x3_deri = -x3 + 0.5 * x4 - 0.1 * x1 ** 2
    # x4_deri = -0.5 * x3 - x4
    # x5_deri = -x5 + 0.5 * x6
    # x6_deri = -0.5 * x5 - x6 + 0.1 * x2 ** 2

    # x7_deri = -x7 + 0.5 * x8
    # x8_deri = -0.5 * x7 - x8
    

    # x1_deri = x2
    # x2_deri = - 5 * sym.sin(x1) - 0.1 * x2
    
    x1_deri = x2
    x2_deri = -x1 - (1 - x1 ** 2) * x2

    # x1_deri = x2
    # x2_deri = - 1 * sym.sin(x1)*sym.cos(x1) - x2 - 1 * sym.sin(x3)*sym.cos(x3)
    # x3_deri = x2 - x3

    # x1_deri = x2
    # x2_deri = - 1 * x1 - x2 - 1 * x3
    # x3_deri = x2 - x3

    # x1_deri = -x1 + 0.5 * x2 - 0.1 * x3 ** 2
    # x2_deri = -0.5 * x1 - x2
    # x3_deri = -x3 + 0.5 * x4 - 0.1 * x1 ** 2
    # x4_deri = -0.5 * x3 - x4
    

    
    # x1_deri = x2
    # x2_deri = - 1 * sym.sin(x1)*sym.cos(x1) - x2 - 1 * sym.sin(x3)*sym.cos(x3)
    # x3_deri = x2 - x3

    # x1_deri = sym.sin(x2)
    # x2_deri = -sym.cos(x2) / (1 - x1)
    # x2_deri = -x2 - 1 * (sym.sin(x2) / (x2)) *x1

    # x1_deri = -x1 + 0.5 * x2 - 0.1 * x9 ** 2
    # x2_deri = -0.5 * x1 - x2
    # x3_deri = -x3 + 0.5 * x4 - 0.1 * x1 ** 2
    # x4_deri = -0.5 * x3 - x4
    # x5_deri = -x5 + 0.5 * x6 + 0.1 * x7 ** 2
    # x6_deri = -0.5 * x5 - x6
    # x7_deri = -x7 + 0.5 * x8
    # x8_deri = -0.5 * x7 - x8
    # x9_deri = -x9 + 0.5 * x10
    # x10_deri = -0.5 * x9 - x10 + 0.1 * x2 ** 2

    # x1_deri = x4 - (x4 + x5 + x6) / 3
    # x2_deri = x5 - (x4 + x5 + x6) / 3
    # x3_deri = x6 - (x4 + x5 + x6) / 3

    # x4_deri = -2 * x4 - sym.sin(x1 - x2) - sym.sin(x1 - x3)
    # x5_deri = -2 * x5 - sym.sin(x2 - x1) - sym.sin(x2 - x3)
    # x6_deri = -2 * x6 - sym.sin(x3 - x1) - sym.sin(x3 - x2)

    # x1_deri = -x1 + 0.5 * x2 - 0.1 * x5 ** 2
    # x2_deri = -0.5 * x1 - x2
    # x3_deri = -x3 + 0.5 * x4 - 0.1 * x1 ** 2
    # x4_deri = -0.5 * x3 - x4
    # x5_deri = -x5 + 0.5 * x6 + 0.1 * x7 ** 2
    # x6_deri = -0.5 * x5 - x6
    # x7_deri = -x7 + 0.5 * x8
    # x8_deri = -0.5 * x7 - x8 - 0.1 * x4 ** 2

    # x1_deri = -x1 + 0.5 * x2 - 0.1 * x7 ** 2
    # x2_deri = -0.5 * x1 - x2
    # x3_deri = -x3 + 0.5 * x4 + 0.1 * x5 ** 2
    # x4_deri = -0.5 * x3 - x4
    # x5_deri = -x5 + 0.5 * x6 
    # x6_deri = -0.5 * x5 - x6
    # x7_deri = -x7 + 0.5 * x8
    # x8_deri = -0.5 * x7 - x8 + 0.1 * x2 ** 2

    return ((-1) * (v.diff(x1) * x1_deri + v.diff(x2) * x2_deri))#  + v.diff(x3) * x3_deri + v.diff(x4) * x4_deri+ v.diff(x5) * x5_deri + v.diff(x6) * x6_deri+ v.diff(x7) * x7_deri + v.diff(x8) * x8_deri  # # ))#  # + v.diff(x9) * x9_deri+ v.diff(x10) * x10_deri


def find_root(func):
    # root = optimize.fsolve(func, guess)
    # outer_root = False

    bounds = Bounds([-1.0, -1.0], [1.0, 1.0])

    result = shgo(func, bounds, n = 1024, iters = 3, sampling_method = "simplicial")
    root = result.x


    if abs(root[0]) > 1.5:
        outer_root = True
        outer_root = True
    
    # elif abs(root[2]) > 1:
        # outer_root = True
    
    # elif abs(root[3]) > 1:
        # outer_root = True
    '''
    elif abs(root[4]) > 1:
        outer_root = True
    elif abs(root[5]) > 1:
        outer_root = True
    '''
        
    # res = func(root)
    # res_success = (np.abs(res[0])<=0.001 and ((np.linalg.norm(root)>0.001) and (not outer_root)))
    # if np.linalg.norm(guess)==0:
        # res_success = ((np.abs(res[0])<=0.001 and (np.linalg.norm(root)<0.001)) or (np.abs(res[0])>0.0001))
    res_success = True
    return root, res_success

def counter_exp_finder_deri(root1, func1, root2, func2, num=400):
    counter_example = []
    pd_counter_example = []
    # distance = np.linspace(np.array([-0.5,-0.5, -0.5]),np.array([0.5,0.5, 0.5]),num)
    distance = np.random.uniform(0, 0.25, (num,2)) # 0.3
    # for i in distance:
        # for j in range(len(i)):
            # i[j] += np.random.randn()

    distance = np.concatenate((distance,np.random.randn(distance.shape[0],distance.shape[1])*0.01),axis=0)
    for j in range(num*2):
        root1_minus = root1 - distance[j]
        root1_minus = np.clip(root1_minus, -1.0, 1.0)
        root1_plus = root1 + distance[j]
        root1_plus = np.clip(root1_plus, -1.0, 1.0)
        root2_minus = root2 - distance[j]
        root2_minus = np.clip(root2_minus, -1.0, 1.0)
        root2_plus = root2 + distance[j]
        root2_plus = np.clip(root2_plus, -1.0, 1.0)

        value1 = func1(root1_minus)
        value2 = func1(root1_plus)
        value3 = func2(root2_minus)
        value4 = func2(root2_plus)
        
        if value1 < 0:
            '''
            if np.sum([i == 0 for i in root1_minus]):
                if value1[0] < 0:
                    pd_counter_example.append((root1_minus).copy())
                else:
                    pd_counter_example.append((root1_minus).copy())
            '''
            pd_counter_example.append((root1_minus).copy())

        if value2 < 0:
            '''
            if np.sum([i == 0 for i in root1_plus]):
                if value1[0] < 0:
                    pd_counter_example.append((root1_plus).copy())
                else:
                    pd_counter_example.append((root1_plus).copy())
            '''
            pd_counter_example.append((root1_plus).copy())
        
        if value3 < - 0:
            '''
            if np.sum([i == 0 for i in root2_minus]):
                if value3 < - 0.0001:
                    counter_example.append((root2_minus).copy())
            else:
                if value3[0] < - 0.0001:
                    counter_example.append((root2_minus).copy())
            '''
            counter_example.append((root2_minus).copy())
        if value4 < - 0:
            '''
            if np.sum([i == 0 for i in root2_plus]):
                if value4[0] < - 0.0001:
                    counter_example.append((root2_plus).copy())
            else:
                if value4[0] < - 0.0001:
                    counter_example.append((root2_plus).copy())
            '''
            counter_example.append((root2_plus).copy())
    del distance

    if func1(root1) < 0:
        pd_counter_example.append(root1.copy())
    if func2(root2) < 0:
        counter_example.append((root2.copy()))
    return counter_example, pd_counter_example

def check_options(sympy_expr):
    
    x1 = sym.symbols('x1')
    x2 = sym.symbols('x2')

    x3, x4, x5, x6 = sym.symbols('x3, x4, x5, x6')

    x7, x8, x9, x10 = sym.symbols("x7, x8, x9, x10")

    origin = float(sympy_expr.evalf(subs = {"x1": 0, "x2": 0}))
    sympy_expr = sympy_expr - origin
    # sympy_expr = sympy_expr.subs(x3, 9.81)
    numpy_expr = sym.lambdify((x1,x2), sympy_expr, "numpy")
    v_dot = derivative_calculate(sympy_expr,x1,x2)
    numpy_v_dot = sym.lambdify((x1,x2), v_dot, "numpy")
        
    function1 = lambda x: numpy_expr(x[0], x[1])
    function2 = lambda x: numpy_v_dot(x[0], x[1])


    root_1, root_1_sucs = find_root(function1)
    root_2, root_2_sucs = find_root(function2)

    counter_example, pd = counter_exp_finder_deri(root_1, function1, root_2, function2)
    counter_exp = counter_example
    counter_exp = np.array(counter_exp)

    
    if ((len(counter_exp) + len(pd)) == 0): 
        valid = True
        return valid, counter_exp
    else:
        valid = False
        return valid, counter_exp
    

def final_check(sympy_expr):
    x1 = sym.symbols('x1')
    x2 = sym.symbols('x2')

    x3, x4, x5, x6 = sym.symbols('x3, x4, x5, x6')

    x7, x8, x9, x10 = sym.symbols("x7, x8, x9, x10")

    origin = float(sympy_expr.evalf(subs = {"x1": 0, "x2": 0}))
    sympy_expr = sympy_expr - origin
    # sympy_expr = sympy_expr.subs(x3, 9.81)
    numpy_expr = sym.lambdify((x1,x2), sympy_expr, "numpy")
    v_dot = derivative_calculate(sympy_expr,x1,x2)
    numpy_v_dot = sym.lambdify((x1,x2), v_dot, "numpy")

    
    # whole state random check
    global function_check_1
    global function_check_2

    def function_check_1(x):
        return numpy_expr(x[0], x[1])
    
    def function_check_2(x):
        return numpy_v_dot(x[0], x[1])
    
    # pgd check
    pgd_check_result = True
    for i in range(5):
        valid, counter_exp = pgd_check(sympy_expr, torch.ones((1000000, 2)))
        if not valid:
            pgd_check_result = False
            print(counter_exp)
            break
    
    if not pgd_check_result:
        return False
    '''
    num_points = 10 ** 7
    # pool_1 = None
    for i in range(5):
        # data = np.random.uniform(-1.0, 1.0, (1 * 10 ** 7, 6))

        boundary_points = []

        # Iterate over each dimension to create points near the boundaries
        for j in range(2):
            # Points near the -1 boundary for the i-th dimension
            point_set_low = np.random.uniform(-1.0, 1.0, (num_points, 2))
            point_set_low[:, j] = -1 + np.abs(0.001 * np.random.rand(num_points))
    
            # Points near the 1 boundary for the i-th dimension
            point_set_high = np.random.uniform(-1.0, 1.0, (num_points, 2))
            point_set_high[:, j] = 1 -  np.abs(0.001 * np.random.rand(num_points))
    

            point_uniform = np.random.uniform(-1.0, 1.0, (num_points, 2))
            # Add these points to the boundary_points list
            boundary_points.append(point_set_low)
            boundary_points.append(point_set_high)
            boundary_points.append(point_uniform)

        # Combine all boundary points into one array
        boundary_points = np.vstack(boundary_points)

        check_1 = function_check_1(boundary_points)
        check_2 = function_check_2(boundary_points)

        if any(check_1 < 0) or any(check_2 < 0):
            return False
        
    return True
    '''
def pgd_attack(
    x0, f, eps, steps=10, lower_boundary=None, upper_boundary=None, direction="minimize"
):
    """
    Use adversarial attack (PGD) to find violating points.
    Args:
      x0: initialization points, in [batch, state_dim].
      f: function f(x) to find the worst case x to maximize.
      eps: perturbation added to x0.
      steps: number of pgd steps.
      lower_boundary: absolute lower bounds of x.
      upper_boundary: absolute upper bounds of x.
    """
    # Set all parameters without gradient, this can speedup things significantly
    grad_status = {}
    try:
        for p in f.parameters():
            grad_status[p] = p.requires_grad
            p.requires_grad_(False)
    except:
        pass

    step_size = eps / steps * 2
    noise = torch.randn_like(x0) * step_size
    if lower_boundary is not None:
        lower_boundary = torch.max(lower_boundary, x0 - eps)
    else:
        lower_boundary = x0 - eps
    if upper_boundary is not None:
        upper_boundary = torch.min(upper_boundary, x0 + eps)
    else:
        upper_boundary = x0 + eps
    x = x0.detach().clone().requires_grad_()
    # Save the best x and best loss.
    best_x = torch.clone(x).detach().requires_grad_(False)
    fill_value = float("-inf") if direction == "maximize" else float("inf")
    best_loss = torch.full(
        size=(x.size(0),),
        requires_grad=False,
        fill_value=fill_value,
        device=x.device,
        dtype=x.dtype,
    )
    for i in range(steps):
        output = f(x1 = x[:,[0]], x2 = x[:,[1]]).squeeze(1).squeeze(1)
        # output = torch.clamp(f(x).squeeze(1), max=0)
        output.mean().backward()
        if direction == "maximize":
            improved_mask = output >= best_loss
        else:
            improved_mask = output <= best_loss
        best_x[improved_mask] = x[improved_mask]
        best_loss[improved_mask] = output[improved_mask]
        # print(f'step = {i}', output.view(-1).detach())
        # print(x.detach(), best_x)
        noise = torch.randn_like(x0) * step_size / (i + 1)
        if direction == "maximize":
            x = (
                (
                    torch.clamp(
                        x + torch.sign(x.grad) * step_size + noise,
                        min=lower_boundary,
                        max=upper_boundary,
                    )
                )
                .detach()
                .requires_grad_()
            )
        else:
            x = (
                (
                    torch.clamp(
                        x - torch.sign(x.grad) * step_size + noise,
                        min=lower_boundary,
                        max=upper_boundary,
                    )
                )
                .detach()
                .requires_grad_()
            )

    # restore the gradient requirement for model parameters
    try:
        for p in f.parameters():
            p.requires_grad_(grad_status[p])
    except:
        pass
    return best_x
    

def pgd_check(sympy, X):

    if len(list(sympy.free_symbols)) < 2:
        return False, np.array([])

    x1, x2, x3, x4, x5, x6, x7, x8 = sym.symbols('x1, x2, x3, x4, x5, x6, x7, x8')

    sympy = derivative_calculate(sympy, x1, x2)

    if len(list(sympy.free_symbols)) < 1:
        return False, np.array([])
    
    sympy_torch = sympytorch.SymPyModule(expressions=[sympy]).to("cuda:1")
    data = torch.rand((X.shape[0], X.shape[1])).to("cuda:1")
    result = pgd_attack(data, sympy_torch, 0.8, steps=30, lower_boundary=torch.tensor([-1.0, -1.0]).to("cuda:1"), upper_boundary=torch.tensor([1.0, 1.0]).to("cuda:1"), direction="minimize")
    result = torch.clamp(result, -1.0, 1.0)
    evaluation = sympy_torch(x1 = result[:,[0]], x2 = result[:,[1]]).squeeze(1).squeeze(1)
    result = result[evaluation < - 1e-15]
    print(sympy_torch(x1 = result[:,[0]], x2 = result[:,[1]]).squeeze(1).squeeze(1))

    return (len(result) == 0), result.cpu().detach().numpy()

In [16]:
x1, x2 = sym.symbols("x1, x2")

V = (x1 ** 2 + x2 ** 2) ** 8

check_options(V)

(True, array([], dtype=float64))

In [17]:
final_check(V)

tensor([], device='cuda:1', grad_fn=<SqueezeBackward1>)
tensor([], device='cuda:1', grad_fn=<SqueezeBackward1>)
tensor([], device='cuda:1', grad_fn=<SqueezeBackward1>)
tensor([], device='cuda:1', grad_fn=<SqueezeBackward1>)
tensor([], device='cuda:1', grad_fn=<SqueezeBackward1>)
